In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd

/home/ubuntu/.conda/envs/watchdog/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('/home/ubuntu/Otree_Project/models/online_shopping_dataset_60model')
model = AutoModelForSequenceClassification.from_pretrained('/home/ubuntu/Otree_Project/models/online_shopping_dataset_60model', num_labels=2).to("cuda")
# 预测函数 输出分类概率
def get_prediction(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return np.array(probs.tolist())

In [3]:
df = pd.read_excel('/home/ubuntu/Otree_Project/Co-Learning/co_learning/情感分析_20220915.xlsx')
text = df['标题/微博内容'].iloc[:100].to_list()

In [4]:
import torch
torch.cuda.empty_cache()

In [6]:
from collections import Counter
from lime.lime_text import LimeTextExplainer
class_names = ['消极', '积极']
explainer = LimeTextExplainer(class_names=class_names)
# 数据集
for i in range(96,100):
    exp = explainer.explain_instance(text[i], get_prediction, num_samples=40, labels=(1,))
    exp.save_to_file('/home/ubuntu/Otree_Project/lime_html/lime_exp%d.html' % i,labels=(1,),predict_proba=False,show_predicted_value=False)